# Creating .txt input files to make the hst stamps for the plots

## Import section

In [7]:
from astropy.io.votable import is_votable, parse
from stage2 import *

from sys import exit

import multiprocessing
from multiprocessing import Manager

## Defining filenames and paths

In [8]:
pathdata  = "../outputs/SelectedGals_sep_by_cluster/"
nameBegin = "matching_fieldsGals_Cassata_and_Zurich_corrected_radius_"
listNumb  = [114, 23, 26, 28, 30, 32, 34, 51, 61, 79, 84, '84-N']
CGrNames  = [i+str(j) for i, j in zip(['CGr']*12, listNumb)]
fullNames = [pathdata + i + '/' + nameBegin + i + '.vot' for i in CGrNames]

## Checking it is a VOtable

In [9]:
for name, ls in zip(fullNames, listNumb):
    voTag = is_VOtable(name)
    if voTag:
        table = parse(name)
        full  = table.get_first_table()
        
        if ls == 84:
            print(full.array['ID'])
        
        print("Size of", name, "is", full.array.shape[0], "\n")
    else:
        exit("Exiting")

The file ../outputs/SelectedGals_sep_by_cluster/CGr114/matching_fieldsGals_Cassata_and_Zurich_corrected_radius_CGr114.vot is a VOtable, right ? True
Size of ../outputs/SelectedGals_sep_by_cluster/CGr114/matching_fieldsGals_Cassata_and_Zurich_corrected_radius_CGr114.vot is 5 

The file ../outputs/SelectedGals_sep_by_cluster/CGr23/matching_fieldsGals_Cassata_and_Zurich_corrected_radius_CGr23.vot is a VOtable, right ? True
Size of ../outputs/SelectedGals_sep_by_cluster/CGr23/matching_fieldsGals_Cassata_and_Zurich_corrected_radius_CGr23.vot is 9 

The file ../outputs/SelectedGals_sep_by_cluster/CGr26/matching_fieldsGals_Cassata_and_Zurich_corrected_radius_CGr26.vot is a VOtable, right ? True
Size of ../outputs/SelectedGals_sep_by_cluster/CGr26/matching_fieldsGals_Cassata_and_Zurich_corrected_radius_CGr26.vot is 10 

The file ../outputs/SelectedGals_sep_by_cluster/CGr28/matching_fieldsGals_Cassata_and_Zurich_corrected_radius_CGr28.vot is a VOtable, right ? True
Size of ../outputs/SelectedGa

## Looping over files, keeping relevant data, renaming some columns and saving into a .txt file

In [16]:
def parsing_and_converting_files(fname, outname, overwrite=False, delimiter=None,
                                 form={'ID':'%3s', 'ID_LAIGLE':'%6s', 'z':'%.5f',
                                       'flag':'%1d', 'ra':'%3.6f', 'dec':'%3.6f', 'Iab':'%2.6f'},
                                 fields=['ID', 'ID_Laigle_16', 'Z_MUSE',
                                         'CONFID', 'RA', 'DEC', 'ip_MAG_ISO'],
                                 newFieldNames={'ID_Laigle_16':'ID_LAIGLE', 'Z_MUSE':'z', 'CONFID':'flag',
                                                'RA':'ra', 'DEC':'dec', 'ip_MAG_ISO':'Iab'}):
    
    #retrieving data
    catalog = Table(parse(fname).get_first_table().array)[fields]
    
    #Change gals names from the VOtable to that found in the folders
    #Only CGr84 has different names in the files and in the folders
    if fname.split("_")[-1] == "CGr84.vot":
        
        matchingFile = '../data/catalogues/CGR84-83_FD+North_zcatalog_withLaigle+16_withFAST_withnewPLATEFIT_totalflux_withnewz_jan19_withFOF_withGALFIT_withGALKIN_jan19.vot'
        if is_VOtable(matchingFile):
            
            #construct a mask which keeps galaxies in group CGr84
            tmpCOSMOSID   = parse(matchingFile).get_first_table().array['COSMOS_Group_Number']
            m             = tmpCOSMOSID==84.0
            
            #the corresponding MUSE id in the .vot file
            CatalogMUSEID = parse(matchingFile).get_first_table().array['ID'][m]
            
            #the MUSE id found in the folders
            FolderMUSEID  = parse(matchingFile).get_first_table().array['ID_in_MUSE_Field'][m]
        else:
            raise Exception("The file", matchingFile, "does not seem to be a VOtable.")

        newGalNumber = np.copy(catalog['ID'])
        for pos, gal in enumerate(catalog['ID']):
            newGalNumber[pos] = FolderMUSEID[CatalogMUSEID==gal][0]
        print("New Gal number :", newGalNumber)
        print("Old Gal number :", catalog['ID'])
        catalog['ID'] = newGalNumber
        print("Old Gal number :", catalog['ID'])
    
    
    #renaming
    for field in newFieldNames.keys():
        catalog.rename_column(field, newFieldNames[field])
     
    IabMask = np.where(catalog['Iab'].mask)
    if np.shape(IabMask)[1] != 0:
        catalog["Iab"][IabMask] = 0.0
    
    #writing
    catalog.write(outname, format='ascii.fixed_width', formats=form, 
                  overwrite=overwrite, delimiter=None)

for file, cg in zip(fullNames, CGrNames):
    overwrite=True
    proc = multiprocessing.Process(name=cg, target=parsing_and_converting_files, 
                                   args=(file, '../data/hst/' + cg + '/catalog_field_galaxies_'  + cg + '.txt',
                                         overwrite, ))
    proc.start()

The file ../data/catalogues/CGR84-83_FD+North_zcatalog_withLaigle+16_withFAST_withnewPLATEFIT_totalflux_withnewz_jan19_withFOF_withGALFIT_withGALKIN_jan19.vot is a VOtable, right ? True
New Gal number : [ 39  54 126 132 146  98]
Old Gal number :  ID
---
220
235
307
313
324
279
Old Gal number :  ID
---
 39
 54
126
132
146
 98
